In [15]:
import pandas as pd
import numpy as np
import math 
import statistics
from pandas import Timestamp
import math

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [16]:
# get stat for -old or new and then compare
def get_stat_for(file, authors, label='-new', extra='', time_a = 1, time_b = 1, time_c = 1):
    pd.set_option("display.max_columns", None)
    df = pd.read_csv(file)
 
    count = 0
    total = 0

    for (index, row) in df.iterrows():
        # calcualte total scores
        total_scores = {'history': 0, 'api': 0, 'code': 0, 'api_s': 0}

        for i in range(1, 11):
            h = row['history_at_' + str(i) + '_v']
            c = row['code_at_' + str(i) + '_v']
            a = row['api_at_' + str(i) + '_v']

            if str(h) != 'nan':
                total_scores['history'] += h
            if str(c) != 'nan':
                total_scores['code'] += c
            if str(a) != 'nan':
                total_scores['api'] += a
                
        # calcualte and rank developer scores
        scores = {}    

        for i in range(1, 11):
            h = row['history_at_' + str(i) + '_v']
            h_developer = row['history_at_' + str(i)]
            c = row['code_at_' + str(i) + '_v']
            c_developer = row['code_at_' + str(i)]
            a = row['api_at_' + str(i) + '_v']
            a_developer = row['api_at_' + str(i)]

            if str(h_developer) != 'nan':
                if str(h) == 'nan' or h == 0 or total_scores['history'] <= 0:
                    scores[h_developer] = scores.get(h_developer, []) + [0]
                else:
                    scores[h_developer] = scores.get(h_developer, []) + [time_a * float(h * 100 / total_scores['history'])]

            if str(c_developer) != 'nan':
                if str(c) == 'nan' or c == 0 or total_scores['code'] <= 0:
                    scores[c_developer] = scores.get(c_developer, []) + [0]
                else:
                    scores[c_developer] = scores.get(c_developer, []) + [time_b * float(c * 100 / total_scores['code'])]


            if str(a_developer) != 'nan':
                if str(a) == 'nan' or a == 0 or total_scores['api'] <= 0:
                    scores[a_developer] = scores.get(a_developer, []) + [0]
                else:
                    #row['jaccard_score'] = row['jaccard_score'] * 2 / i
                    scores[a_developer] = scores.get(a_developer, []) + [time_c * row['jaccard_score'] * float(a * 100 / total_scores['api'])]
                    pass
                

        # calculate the total
        for index_, score in scores.items():
            if len(score) == 0:
                scores[index_] = 0
            else:
                scores[index_] = sum(score)  ## ** (1/float(len(score)))


        sort = sorted(scores.items(), key=lambda x: x[1], reverse=True)

        chosen = ''
        if len(sort) != 0:
            chosen = sort[0][0]
        if chosen == row['assignees']:
            count += 1
        total += 1   

    if extra == 'ratio':
        return count / total
    print(label)
    print(count)
    print(count / total)
    print(total)
    return authors

In [17]:
# def calculate_score_absba(history_score, code_score, api_score):
#     ave = 0
#     for try_ in range(0, 5):
#         answer = get_stat_for('../data/output/try1/jdt_direct_' + str(try_) +'_.csv', {}, 
#                               '-new', 'ratio', 1, 1, 1)
#         ave += answer
#     print(ave / 5)
#     return ave

def calculate_score_bsba(history_score, code_score, api_score):
    x = get_stat_for('../data/output/jdt_direct.csv', {}, 
                          '-new', 'ratio', history_score, code_score, api_score)
    print(x)
    return x

In [18]:
parameter_range = [0, 0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.55, 0.6, 0.7, 0.75, 0.8, 0.9, 0.99, 0.999, 1]

# answer = {}
# for history_score in parameter_range:
#     for code_score in parameter_range:
#         #for api_score in parameter_range:
#         api_score = 0
#         key = str(history_score) + '_' + str(code_score) + '_' + str(api_score)
#         value = calculate_score_bsba(history_score, code_score, api_score)
#         print('done: ' + key)
#         answer[key] = value


answer = {}
for history_score in parameter_range:
    for code_score in parameter_range:
        for api_score in parameter_range:
            key = str(history_score) + '_' + str(code_score) + '_' + str(api_score)
            value = calculate_score_bsba(history_score, code_score, api_score)
            print('done: ' + key)
            answer[key] = value

0.47
done: 0_0_0
0.47
done: 0_0_0.05
0.47
done: 0_0_0.1
0.47
done: 0_0_0.2
0.47
done: 0_0_0.25
0.47
done: 0_0_0.3
0.47
done: 0_0_0.4
0.47
done: 0_0_0.5
0.47
done: 0_0_0.55
0.47
done: 0_0_0.6
0.47
done: 0_0_0.7
0.47
done: 0_0_0.75
0.47
done: 0_0_0.8


KeyboardInterrupt: 

In [14]:
max_= max(answer, key= lambda x: answer[x])
print(answer[max_])
print(max_)

#print(answer['0.99_0.75_0'])
#print(answer['0.7_0.5_0.8'])
#print(answer['0.7_0.5_0.25'])

#print(answer['1_1_0'])

0.47833333333333333
0.05_0.05_0.05


In [19]:
test_dict = answer
f = open("dict-absba-without.txt", "w")
f.write("{\n")
for k in test_dict.keys():
    f.write("'{}':'{}'\n".format(k, test_dict[k]))
f.write("}")
f.close()